## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

## Loading Dataset

In [3]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [4]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [5]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [6]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [7]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
validation_dataset = validation_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

### Tokenizing Data

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT, return_tensors="pt")
  tokenized_batch["labels"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [10]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 7013/7013 [00:01<00:00, 6950.45 examples/s]


#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [29]:
from custom_model import AutoCompositeModel

model = AutoCompositeModel(device).to(device)

In [30]:
input = torch.tensor(tokenized_test_dataset['input_ids'][:10]).float().to(device)
input.dtype

torch.float32

## Training Setup

### Data Loaders

In [31]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=16, shuffle=True)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=16, shuffle=True)

### Loss Function

In [32]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [33]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-5, momentum=0.8)

In [34]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [35]:
from training_own import Trainer, TrainerConfiguration, get_model_params

config = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)

In [36]:
get_model_params(model)

1777989

In [37]:
trainer = Trainer(model=model, trainer_configuration=config, input_column='input_ids', output_column='labels')

In [38]:
len(training_loader)

6137

In [39]:
trainer.train_many_epochs(epochs=8, logging_frequency=300, evaluate_when_logging=False)

EPOCH 1:


 batch 300 training_loss: 1.5919880652427674 training_accuracy: 0.38104167580604553
 batch 600 training_loss: 1.5863707367579143 training_accuracy: 0.49166667461395264
 batch 900 training_loss: 1.5808382944266002 training_accuracy: 0.49041667580604553
 batch 1200 training_loss: 1.5758536382516226 training_accuracy: 0.4985416829586029
 batch 1500 training_loss: 1.5684045406182607 training_accuracy: 0.5062500238418579
 batch 1800 training_loss: 1.56304585536321 training_accuracy: 0.5014583468437195
 batch 2100 training_loss: 1.5569155955314635 training_accuracy: 0.5110417008399963
 batch 2400 training_loss: 1.5525289980570476 training_accuracy: 0.5054166913032532
 batch 2700 training_loss: 1.5490184779961904 training_accuracy: 0.4931250214576721
 batch 3000 training_loss: 1.5403145523866018 training_accuracy: 0.5116666555404663
 batch 3300 training_loss: 1.5388207133611043 training_accuracy: 0.49979168176651
 batch 3600 training_loss: 1.5349925100803374 training_accuracy: 0.4910416901111